# Calorie expenditure during road cycling

In this project I'll try to answer two questions:

    - What variables affect calorie expenditure the most when we're cycling?
    - Can this caloric expenditure be accurately predicted for any given route?
    
To answer them I'll be using a dataset of my cycling rides from 2016 up to this day, courtesy of **SportTracks**'s elegantly simple data export options.

## 1. Data wrangling

Since our data is in csv format, we'll begin our data exploration by loading it into a dataframe object.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('workouts.csv', encoding='utf-8', )

In [3]:
df.head()

,Nombre,Inicio,Deporte,Distancia,Duración,Velocidad media,Calorías,Calorías (/hora),Pulso promedio,Aumento del Desnivel,Temperatura
0,Cycling: Road,2021-05-30 11:09:26,Carretera,"20,04 km",42:26:00,"28,3 km/h",506.0,715.0,144.0,152,"22,3"
1,Cycling: Road,2021-05-28 8:51:04,Carretera,"100,29 km",3:21:59,"29,8 km/h",1799.0,534.0,131.0,610,"16,6"
2,Cycling: Road,2021-05-26 8:51:07,Carretera,"124,78 km",4:44:16,"26,3 km/h",2741.0,579.0,140.0,1751,"15,9"
3,Cycling: Road,2021-05-24 10:46:51,Carretera,"36,25 km",1:18:23,"27,7 km/h",643.0,492.0,124.0,186,"16,2"
4,Cycling: Mountain,2021-05-23 9:20:16,Montaña,"78,61 km",3:10:01,"24,8 km/h",2031.0,641.0,143.0,381,"13,9"


In [4]:
#Let's check for missing values. Since very few activities have missing data we can safely drop them.

df.isnull().sum()

Nombre                  0
Inicio                  0
Deporte                 0
Distancia               0
Duración                0
Velocidad media         4
Calorías                4
Calorías (/hora)        4
Pulso promedio          8
Aumento del Desnivel    0
Temperatura             3
dtype: int64

In [5]:
#Dropping the rows with missing data.

df.dropna(axis=0, how='any', inplace=True)

In [6]:
#To clean up our dataframe and prevent any future naming issues, let's rename the columns.

df.rename(columns={'Nombre':'name',
                      'Inicio':'start',
                      'Deporte': 'sport',
                      'Distancia':'distance',
                      'Duración':'duration',
                      'Velocidad media':'avg_speed',
                      'Calorías':'calories',
                      'Calorías (/hora)':'cals_per_hour',
                      'Pulso promedio':'heartrate',
                      'Aumento del Desnivel':'climb',
                      'Temperatura':'temp'}, inplace=True)

In [7]:
df.head()

,name,start,sport,distance,duration,avg_speed,calories,cals_per_hour,heartrate,climb,temp
0,Cycling: Road,2021-05-30 11:09:26,Carretera,"20,04 km",42:26:00,"28,3 km/h",506.0,715.0,144.0,152,"22,3"
1,Cycling: Road,2021-05-28 8:51:04,Carretera,"100,29 km",3:21:59,"29,8 km/h",1799.0,534.0,131.0,610,"16,6"
2,Cycling: Road,2021-05-26 8:51:07,Carretera,"124,78 km",4:44:16,"26,3 km/h",2741.0,579.0,140.0,1751,"15,9"
3,Cycling: Road,2021-05-24 10:46:51,Carretera,"36,25 km",1:18:23,"27,7 km/h",643.0,492.0,124.0,186,"16,2"
4,Cycling: Mountain,2021-05-23 9:20:16,Montaña,"78,61 km",3:10:01,"24,8 km/h",2031.0,641.0,143.0,381,"13,9"


In [8]:
#The 'name column' doesn't give us any meaningful information so we'll drop it.

df.drop('name',axis=1, inplace=True)
df.head()

,start,sport,distance,duration,avg_speed,calories,cals_per_hour,heartrate,climb,temp
0,2021-05-30 11:09:26,Carretera,"20,04 km",42:26:00,"28,3 km/h",506.0,715.0,144.0,152,"22,3"
1,2021-05-28 8:51:04,Carretera,"100,29 km",3:21:59,"29,8 km/h",1799.0,534.0,131.0,610,"16,6"
2,2021-05-26 8:51:07,Carretera,"124,78 km",4:44:16,"26,3 km/h",2741.0,579.0,140.0,1751,"15,9"
3,2021-05-24 10:46:51,Carretera,"36,25 km",1:18:23,"27,7 km/h",643.0,492.0,124.0,186,"16,2"
4,2021-05-23 9:20:16,Montaña,"78,61 km",3:10:01,"24,8 km/h",2031.0,641.0,143.0,381,"13,9"


In [9]:
#Some of our columns need to be typecasted into more useful formats. Let's get to it.

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 813
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   start          800 non-null    object 
 1   sport          800 non-null    object 
 2   distance       800 non-null    object 
 3   duration       800 non-null    object 
 4   avg_speed      800 non-null    object 
 5   calories       800 non-null    float64
 6   cals_per_hour  800 non-null    float64
 7   heartrate      800 non-null    float64
 8   climb          800 non-null    int64  
 9   temp           800 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 68.8+ KB


In [10]:
#Converting 'start' to datetime format. Using the correct formal is crucial at this point.

df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')

In [11]:
#The column Dtype has been changed successfully.

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 813
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   start          800 non-null    datetime64[ns]
 1   sport          800 non-null    object        
 2   distance       800 non-null    object        
 3   duration       800 non-null    object        
 4   avg_speed      800 non-null    object        
 5   calories       800 non-null    float64       
 6   cals_per_hour  800 non-null    float64       
 7   heartrate      800 non-null    float64       
 8   climb          800 non-null    int64         
 9   temp           800 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 68.8+ KB


In [12]:
#'Sport' is a categorical variable. Let's see the values it can have.

df['sport'].value_counts()

Carretera     538
Montaña       129
Ciclismo       69
Virtual        59
Interiores      4
Entrenador      1
Name: sport, dtype: int64

In [13]:
#Those 6 categories can be summed up into just 3 (road, mountain, indoor).

df['sport'].replace('Carretera','road',inplace=True)
df['sport'].replace('Montaña','mountain',inplace=True)
df['sport'].replace('Ciclismo','road',inplace=True)
df['sport'].replace('Virtual','indoor',inplace=True)
df['sport'].replace('Interiores','indoor',inplace=True)
df['sport'].replace('Entrenador','indoor',inplace=True)

In [14]:
#Let's check if the string replace has worked successfully.

df['sport'].value_counts()

road        607
mountain    129
indoor       64
Name: sport, dtype: int64

In [15]:
#To convert 'distance' into a float we'll simply use string replace.

df['distance'] = df['distance'].str.replace(' km','')
df['distance'] = df['distance'].str.replace(',','.') #Replacing dots with commas.
df['distance'] = df['distance'].astype(float) #Finally typecasting as float.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 813
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   start          800 non-null    datetime64[ns]
 1   sport          800 non-null    object        
 2   distance       800 non-null    float64       
 3   duration       800 non-null    object        
 4   avg_speed      800 non-null    object        
 5   calories       800 non-null    float64       
 6   cals_per_hour  800 non-null    float64       
 7   heartrate      800 non-null    float64       
 8   climb          800 non-null    int64         
 9   temp           800 non-null    object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 68.8+ KB


In [17]:
df.head()

,start,sport,distance,duration,avg_speed,calories,cals_per_hour,heartrate,climb,temp
0,2021-05-30 11:09:26,road,20.04,42:26,"28,3 km/h",506.000,715.0,144.0,152.000,"22,3"
1,2021-05-28 08:51:04,road,100.29,3:21:59,"29,8 km/h",1.799,534.0,131.0,610.000,"16,6"
2,2021-05-26 08:51:07,road,124.78,4:44:16,"26,3 km/h",2.741,579.0,140.0,1.751,"15,9"
3,2021-05-24 10:46:51,road,36.25,1:18:23,"27,7 km/h",643.000,492.0,124.0,186.000,"16,2"
4,2021-05-23 09:20:16,mountain,78.61,3:10:01,"24,8 km/h",2.031,641.0,143.0,381.000,"13,9"


In [17]:
#To convert 'duration' to TimeDelta we must add the missing 00: in front of the shorter routes (less than an hour).

df['duration'] = ['00:'+ x for x in df['duration']]

In [18]:
df.head()

,start,sport,distance,duration,avg_speed,calories,cals_per_hour,heartrate,climb,temp
0,2021-05-30 11:09:26,road,20.04,00:42:26:00,"28,3 km/h",506.0,715.0,144.0,152,"22,3"
1,2021-05-28 08:51:04,road,100.29,00:3:21:59,"29,8 km/h",1799.0,534.0,131.0,610,"16,6"
2,2021-05-26 08:51:07,road,124.78,00:4:44:16,"26,3 km/h",2741.0,579.0,140.0,1751,"15,9"
3,2021-05-24 10:46:51,road,36.25,00:1:18:23,"27,7 km/h",643.0,492.0,124.0,186,"16,2"
4,2021-05-23 09:20:16,mountain,78.61,00:3:10:01,"24,8 km/h",2031.0,641.0,143.0,381,"13,9"


In [19]:
#This function will get rid of surplus characters.

def time_add(x):
    if len(x) == 10:
        return x[3:]
    elif len(x) == 11:
        return x[3:]
    else:
        return x

In [20]:
#Applying the function.

df['duration'] = df['duration'].apply(time_add)

In [21]:
df.head()

,start,sport,distance,duration,avg_speed,calories,cals_per_hour,heartrate,climb,temp
0,2021-05-30 11:09:26,road,20.04,42:26:00,"28,3 km/h",506.0,715.0,144.0,152,"22,3"
1,2021-05-28 08:51:04,road,100.29,3:21:59,"29,8 km/h",1799.0,534.0,131.0,610,"16,6"
2,2021-05-26 08:51:07,road,124.78,4:44:16,"26,3 km/h",2741.0,579.0,140.0,1751,"15,9"
3,2021-05-24 10:46:51,road,36.25,1:18:23,"27,7 km/h",643.0,492.0,124.0,186,"16,2"
4,2021-05-23 09:20:16,mountain,78.61,3:10:01,"24,8 km/h",2031.0,641.0,143.0,381,"13,9"


In [22]:
#Now we can finally convert it to a TimeDelta object.

df['duration'] = pd.to_timedelta(df['duration'])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 813
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   start          800 non-null    datetime64[ns] 
 1   sport          800 non-null    object         
 2   distance       800 non-null    float64        
 3   duration       800 non-null    timedelta64[ns]
 4   avg_speed      800 non-null    object         
 5   calories       800 non-null    float64        
 6   cals_per_hour  800 non-null    float64        
 7   heartrate      800 non-null    float64        
 8   climb          800 non-null    int64          
 9   temp           800 non-null    object         
dtypes: datetime64[ns](1), float64(4), int64(1), object(3), timedelta64[ns](1)
memory usage: 68.8+ KB


In [24]:
#The last column that we need to work on is 'avg_speed'. Let's convert it to a float.

df['avg_speed'] = df['avg_speed'].str.replace(' km/h','')
df['avg_speed'] = df['avg_speed'].str.replace(',','.')
df['avg_speed'] = df['avg_speed'].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 813
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   start          800 non-null    datetime64[ns] 
 1   sport          800 non-null    object         
 2   distance       800 non-null    float64        
 3   duration       800 non-null    timedelta64[ns]
 4   avg_speed      800 non-null    float64        
 5   calories       800 non-null    float64        
 6   cals_per_hour  800 non-null    float64        
 7   heartrate      800 non-null    float64        
 8   climb          800 non-null    int64          
 9   temp           800 non-null    object         
dtypes: datetime64[ns](1), float64(5), int64(1), object(2), timedelta64[ns](1)
memory usage: 68.8+ KB


In [27]:
#Typecasting 'temp'.

df['temp'] = df['temp'].astype(str)
df['temp'] = df['temp'].str.replace(',','.')
df['temp'] = df['temp'].astype(float)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 813
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   start          800 non-null    datetime64[ns] 
 1   sport          800 non-null    object         
 2   distance       800 non-null    float64        
 3   duration       800 non-null    timedelta64[ns]
 4   avg_speed      800 non-null    float64        
 5   calories       800 non-null    float64        
 6   cals_per_hour  800 non-null    float64        
 7   heartrate      800 non-null    float64        
 8   climb          800 non-null    int64          
 9   temp           800 non-null    float64        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1), timedelta64[ns](1)
memory usage: 68.8+ KB


## 2. Data exploration

Now that all our columns are nice and tidy we can finally take a good look at our data.
Let's begin by importing our favourite libraries.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [29]:
df.describe()

,distance,duration,avg_speed,calories,cals_per_hour,heartrate,climb,temp
count,800.000000,800,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,45.076225,0 days 09:10:29.855000,24.776125,1005.875000,553.178750,133.875000,455.420000,17.439125
std,26.918241,0 days 16:54:49.296177675,4.127325,584.823373,74.432517,7.815416,392.482987,6.484857
min,0.080000,0 days 00:00:26,8.100000,2.000000,277.000000,96.000000,0.000000,-0.500000
25%,24.930000,0 days 01:30:18.750000,22.700000,592.500000,500.000000,129.000000,194.000000,12.600000
50%,40.525000,0 days 02:00:44.500000,25.400000,904.500000,549.000000,134.000000,335.500000,16.500000
75%,58.065000,0 days 03:16:52.250000,27.000000,1274.250000,600.000000,139.000000,591.500000,22.100000
max,204.640000,2 days 11:49:00,37.700000,4295.000000,780.000000,160.000000,2602.000000,35.500000
